# News Keywords Extraction

In [ ]:
import pandas as pd
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
import plotly.express as px

from sklearn.feature_extraction.text import TfidfVectorizer

import fasttext

import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import spacy
sp = spacy.load('en_core_web_sm')

import emoji
import string

## Reading Data

In [ ]:
df_tweets = pd.read_json('data/news/news_tweets/2022-03-16__2022-04-01/tweets.json')
df_tweets

In [ ]:
df_authors = pd.read_json('data/news/news_tweets/2022-03-16__2022-04-01/users.json')
df_authors

## Data Exploration

### Counts and IDs

In [ ]:
ids = pd.read_csv('data/news/twitter_users_news.csv', header=None, names=['handle', 'id', 'website'])['id']
ids

In [ ]:
src_count = df_tweets.groupby('author_id')['id'].count()
src_count

In [ ]:
src_count.value_counts()

#### Some users have less thant 90 tweets

In [ ]:
src_count[src_count < 90]

In [ ]:
df_authors[df_authors['id'] == 247723476]

#### Some users are extracted although not asked to be extracted

In [ ]:
df_authors[~df_authors['id'].isin(ids.values)]

In [ ]:
df_tweets[~df_tweets.author_id.isin(ids.values)].author_id.unique()

### Exploring Withheld

In [ ]:
df_tweets[~df_tweets.withheld.isna()].head()

In [ ]:
df_authors[df_authors['id'].isin(df_tweets[~df_tweets.withheld.isna()].author_id.unique())]

In [ ]:
df_authors[~df_authors.withheld.isna()]

### Data Language Statistics 

In [ ]:
lang_count = df_tweets.groupby('lang')[['id']].count().reset_index()
lang_count['id'] = lang_count['id'] / lang_count['id'].sum()
fig = px.pie(lang_count, names='lang', values='id', title='Language proportions'.title(), height=700, hover_name=(lang_count['id']*100).round(2).apply(str) + '%')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide', title_x=0.5,)
fig.show()

### Language Verification

In [ ]:
df_english = df_tweets[df_tweets.lang == 'en'].copy()
df_english

In [ ]:
df_english.text[:5].tolist()

## Language Identification

In [ ]:
model = fasttext.load_model('data/lid.176.bin')

Remove redundant spaces, links and emojis

In [ ]:
df_tweets['clean_text'] = df_tweets.text.str.replace(r'\s+', ' ', regex=True).str.replace(r'http\S+', '', regex=True).str.strip().apply(lambda s: emoji.replace_emoji(s, replace=''))
df_tweets['clean_text']

In [ ]:
langs = model.predict(df_tweets['clean_text'].tolist(), k=2)

In [ ]:
df = pd.DataFrame(list(zip(*langs)), columns=['pred_lang', 'pred_conf'])
df

In [ ]:
df[['pred_lang', 'pred_lang_2']] = pd.DataFrame(df.pred_lang.values.tolist(), index=df.index)
df['pred_lang'] = df.pred_lang.str.replace('__label__', '')
df['pred_lang_2'] = df.pred_lang_2.str.replace('__label__', '')
df[['pred_conf', 'pred_conf_2']] = pd.DataFrame(df.pred_conf.values.tolist(), index=df.index)
df

In [ ]:
df.pred_conf.hist()

In [ ]:
df_tweets = df_tweets.join(df)
df_tweets

In [ ]:
(df_tweets.lang == df_tweets.pred_lang).value_counts(normalize=False)

In [ ]:
(df_tweets.lang == df_tweets.pred_lang).value_counts(normalize=True)

In [ ]:
df_tweets[df_tweets.pred_conf < 0.5]

In [ ]:
df_tweets = df_tweets[(df_tweets.pred_conf >= 0.5) & (df_tweets.clean_text.apply(len) > 0)]
len(df_tweets)

## English Keywords

In [ ]:
df_english = df_tweets[df_tweets.pred_lang == 'en'].copy()
df_english

### Remove Retweets, Quotes and Replies

In [ ]:
df_english['tweet_type'] = df_english['referenced_tweets'].apply(lambda l: tuple(sorted([d['type'] for d in l])) if type(l) == list else l)
df_english.head()

In [ ]:
df_english.tweet_type.value_counts(dropna=False)

In [ ]:
df_english = df_english[df_english['tweet_type'].isna()]
df_english.head()

### Text Cleaning

In [ ]:
english_texts = df_english.text
english_texts

#### Remove redundant spaces, links and emojis

In [ ]:
english_texts = english_texts.str.replace(r'\s+', ' ', regex=True).str.replace(r'http\S+', '', regex=True).str.strip().apply(lambda s: emoji.replace_emoji(s, replace=''))
english_texts

#### Tokenize

In [ ]:
english_tokens = english_texts.parallel_apply(word_tokenize)
english_tokens

#### Tag Tokens

In [ ]:
english_annotated_tokens = english_tokens.parallel_apply(pos_tag)
english_annotated_tokens

#### Separate Text from Mentions and Hashtags

In [ ]:
def separate(ls):
    res = {'text': [],
           'hashtags': [],
           'mentions': []}
    
    hashtag = False
    mention = False
    
    for s, t in ls:
        if hashtag:
            hashtag = False
            res['hashtags'].append('#' + s)
        elif mention:
            mention = False
            res['mentions'].append('@' + s)
        else:
            if s == '#':
                hashtag = True
            elif s == '@':
                mention = True
            else:    
                res['text'].append((s, t))
            
    return res

In [ ]:
separate(english_annotated_tokens.iloc[0])

In [ ]:
df_english_tokens = pd.DataFrame(english_annotated_tokens.apply(separate).tolist(), index=english_annotated_tokens.index)
df_english_tokens

#### Translate Tags

In [ ]:
df_english_token_texts = df_english_tokens.text.explode().dropna()
df_english_token_texts

In [ ]:
# https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None

In [ ]:
df_english_token_texts = df_english_token_texts.apply(lambda x: (x[0].lower(), pos_tagger(x[1])))
df_english_token_texts

In [ ]:
df_english_tagged_tokens = pd.DataFrame(df_english_token_texts.tolist(), index=df_english_token_texts.index, columns=['token', 'tag'])
df_english_tagged_tokens

#### Remove punctuations

In [ ]:
all_punct_regex ='^([^\w\s]+|_)$'

In [ ]:
df_english_tagged_tokens = df_english_tagged_tokens[(~df_english_tagged_tokens.token.isin(list(string.punctuation))) & (~df_english_tagged_tokens.token.str.match(all_punct_regex))]
df_english_tagged_tokens

Remove all numbers

In [ ]:
df_english_tagged_tokens = df_english_tagged_tokens[(~df_english_tagged_tokens.token.str.isnumeric())]
df_english_tagged_tokens

Remove stopwords

In [ ]:
df_english_tagged_tokens = df_english_tagged_tokens[(~df_english_tagged_tokens.token.str.lower().isin(stopwords.words())) & (~df_english_tagged_tokens.token.str.lower().isin(sp.Defaults.stop_words))]
df_english_tagged_tokens

#### Lemmatize

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize(word, tag):
    if tag is None:
        # if there is no available tag, append the token as is
        return word
    else:       
        # else use the tag to lemmatize the token
        return lemmatizer.lemmatize(word, tag)

In [ ]:
df_english_tagged_tokens = df_english_tagged_tokens.copy()
df_english_tagged_tokens['token'] = df_english_tagged_tokens.parallel_apply(lambda row: lemmatize(row['token'], row['tag']), axis=1)
df_english_tagged_tokens

### Value Counts

In [ ]:
df_english_tagged_tokens['tuple'] = list(zip(df_english_tagged_tokens.token, df_english_tagged_tokens.tag))
df_english_tagged_tokens

In [ ]:
token_counts = df_english_tagged_tokens.tuple.value_counts()
token_counts

In [ ]:
token_counts = token_counts[token_counts > 3]
token_counts

In [ ]:
df_token_counts = token_counts.to_frame(name='count').reset_index().rename(columns={'index': 'token'})
df_token_counts

In [ ]:
df_token_counts[['token', 'tag']] = pd.DataFrame(df_token_counts.token.tolist(), index=df_token_counts.index)
df_token_counts

In [ ]:
df_token_counts.to_csv('dump/token_count.csv')

In [ ]:
df_token_counts.groupby('tag').apply(lambda x: x.nlargest(15, ['count'])).reset_index(drop=True)

In [ ]:
df_token_counts[df_token_counts.token == 'breaking']

### TF-IDF

#### Read Random Tweets

In [ ]:
df_random_tweets = pd.read_parquet('data/news/random_tweets.parquet')
df_random_tweets

#### Clean Random Tweet Texts

##### Language Identification

Remove redundant spaces, links and emojis

In [ ]:
df_random_tweets['clean_text'] = df_random_tweets.text.str.replace(r'\s+', ' ', regex=True).str.replace(r'http\S+', '', regex=True).str.strip().apply(lambda s: emoji.replace_emoji(s, replace=''))
df_random_tweets['clean_text']

In [ ]:
random_langs = model.predict(df_random_tweets['clean_text'].tolist(), k=2)

In [ ]:
df_random_langs = pd.DataFrame(list(zip(*random_langs)), columns=['pred_lang', 'pred_conf'])
df_random_langs

In [ ]:
df_random_langs[['pred_lang', 'pred_lang_2']] = pd.DataFrame(df_random_langs.pred_lang.values.tolist(), index=df_random_langs.index)
df_random_langs['pred_lang'] = df_random_langs.pred_lang.str.replace('__label__', '')
df_random_langs['pred_lang_2'] = df_random_langs.pred_lang_2.str.replace('__label__', '')
df_random_langs[['pred_conf', 'pred_conf_2']] = pd.DataFrame(df_random_langs.pred_conf.values.tolist(), index=df_random_langs.index)
df_random_langs

In [ ]:
df_random_langs.pred_conf.hist()

In [ ]:
df_random_tweets = df_random_tweets.join(df_random_langs)
df_random_tweets

In [ ]:
(df_random_tweets.lang == df_random_tweets.pred_lang).value_counts(normalize=False)

In [ ]:
(df_random_tweets.lang == df_random_tweets.pred_lang).value_counts(normalize=True)

In [ ]:
df_random_tweets = df_random_tweets[(df_random_tweets.pred_conf >= 0.5) & (df_random_tweets.clean_text.apply(len) > 0)]
len(df_random_tweets)

In [ ]:
df_random_english = df_random_tweets[df_random_tweets.pred_lang == 'en'].copy()
df_random_english

##### Remove Retweets, Quotes and Replies

In [ ]:
df_random_english['tweet_type'] = df_random_english['referenced_tweets'].apply(lambda l: tuple(sorted([d['type'] for d in l])) if type(l) == list or type(l) == np.ndarray else l)
df_random_english.head()

In [ ]:
df_random_english.tweet_type.value_counts(dropna=False)

In [ ]:
df_random_english = df_random_english[df_random_english['tweet_type'].isna()]
df_random_english.head()

##### Text Cleaning

In [ ]:
random_english_texts = df_random_english.text
random_english_texts

###### Remove redundant spaces, links and emojis

In [ ]:
random_english_texts = random_english_texts.str.replace(r'\s+', ' ', regex=True).str.replace(r'http\S+', '', regex=True).str.strip().apply(lambda s: emoji.replace_emoji(s, replace=''))
random_english_texts

###### Tokenize

In [ ]:
random_english_tokens = random_english_texts.parallel_apply(word_tokenize)
random_english_tokens

###### Tag Tokens

In [ ]:
random_english_annotated_tokens = random_english_tokens.parallel_apply(pos_tag)
random_english_annotated_tokens

###### Separate Text from Mentions and Hashtags

In [ ]:
separate(random_english_annotated_tokens.iloc[0])

In [ ]:
df_random_english_tokens = pd.DataFrame(random_english_annotated_tokens.apply(separate).tolist(), index=random_english_annotated_tokens.index)
df_random_english_tokens

###### Translate Tags

In [ ]:
df_random_english_token_texts = df_random_english_tokens.text.explode().dropna()
df_random_english_token_texts

In [ ]:
df_random_english_token_texts = df_random_english_token_texts.apply(lambda x: (x[0].lower(), pos_tagger(x[1])))
df_random_english_token_texts

In [ ]:
df_random_english_tagged_tokens = pd.DataFrame(df_random_english_token_texts.tolist(), index=df_random_english_token_texts.index, columns=['token', 'tag'])
df_random_english_tagged_tokens

###### Remove punctuations

In [ ]:
all_punct_regex ='^([^\w\s]+|_)$'

In [ ]:
df_random_english_tagged_tokens = df_random_english_tagged_tokens[(~df_random_english_tagged_tokens.token.isin(list(string.punctuation))) & (~df_random_english_tagged_tokens.token.str.match(all_punct_regex))]
df_random_english_tagged_tokens

Remove all numbers

In [ ]:
df_random_english_tagged_tokens = df_random_english_tagged_tokens[(~df_random_english_tagged_tokens.token.str.isnumeric())]
df_random_english_tagged_tokens

Remove stopwords

In [ ]:
df_random_english_tagged_tokens = df_random_english_tagged_tokens[(~df_random_english_tagged_tokens.token.str.lower().isin(stopwords.words())) & (~df_random_english_tagged_tokens.token.str.lower().isin(sp.Defaults.stop_words))]
df_random_english_tagged_tokens

###### Lemmatize

In [ ]:
df_random_english_tagged_tokens = df_random_english_tagged_tokens.copy()
df_random_english_tagged_tokens['token'] = df_random_english_tagged_tokens.parallel_apply(lambda row: lemmatize(row['token'], row['tag']), axis=1)
df_random_english_tagged_tokens

#### Create 2 Documents

In [ ]:
news_doc_list = df_english_tagged_tokens[~df_english_tagged_tokens.token.isna()].token.to_list()
news_doc = ' '.join(news_doc_list)
len(news_doc)

In [ ]:
random_doc_list = df_random_english_tagged_tokens[~df_random_english_tagged_tokens.tag.isna()].token.to_list()
random_doc = ' '.join(random_doc_list)
len(random_doc)

#### TF-IDF

In [ ]:
vect = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, lowercase=False)
tf_idf = vect.fit_transform([news_doc_list, random_doc_list])

In [ ]:
tf_idf.shape

In [ ]:
res = pd.DataFrame(tf_idf.toarray().T, index=vect.get_feature_names_out(), columns=['news', 'not_news'])
res

In [ ]:
res['diff'] = res['news'] - res['not_news']

In [ ]:
res['news'].nlargest(20)

In [ ]:
res['not_news'].nlargest(20)

In [ ]:
res.loc['breaking']

In [ ]:
res['diff'].nlargest(60)

In [ ]:
res['diff'].loc['breaking']

In [ ]:
res['diff'].nlargest(60).reset_index()['index'].to_csv('data/news/news_words.csv', index=None)